<a href="https://colab.research.google.com/github/FabioDamacena/Pandas/blob/main/Data_Cleaning_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Cleaning com Python**

## **Dados missing fora do padrão**

*   Falta do valor
*   Valores truncados
*   Valores do usuário



In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/FabioDamacena/Pandas/main/file_test.csv', sep=';')

In [ ]:
df.head()

Contagem da quantidade de missing values por coluna e total de missing values da base:

In [ ]:
df.isnull().sum()

In [ ]:
df.isnull().sum().sum()

Analisando colunas específicas:

In [ ]:
df.ST_NUM # Pandas identificou o valor vazio como um NaN

In [ ]:
df.NUM_BEDROOM # Na última linha não identificou o 'na' como NaN

In [ ]:
df.NUM_BEDROOM.isnull() # Não contou a última linha como nula

In [ ]:
df.OWN_OCCUPIED # Coluna de strings com valor numérico não foi identificada a irregularidade

Resolvendo as irregularidades:

*   Valores inteiros em strings ou vice e versa
*   Utilidade do cleaning antes de usar o scikit learn para treinar algoritmos



Criação de uma lista de valores missing para que o pandas os identifique e utilização do parâmetro na_values:

In [ ]:
missing_values = ["n/a", "na", "--", "."]

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/FabioDamacena/Pandas/main/file_test.csv', na_values = missing_values, sep = ';')

In [ ]:
df.head(9)

In [ ]:
df.isnull().sum().sum()

Substituição de valores numéricos por strings:

In [ ]:
count = 0
for row in df.OWN_OCCUPIED:
  try:
    int(row) # Tenta converter o valor encontrado na coluna para inteiro, se conseguir, substitui por NaN. Se fosse ao contrário (string em coluna de inteiros), utilizaria str(row)
    df.loc[count, 'OWN_OCCUPIED'] = np.nan # np.nan é atributo que transforma o valor em NaN
  except ValueError: # Continua o loop se der erro
    pass
  count += 1

In [ ]:
df.OWN_OCCUPIED # Substitui o valor numérico por NaN

In [ ]:
df.isnull().sum().sum()

## **Biblioteca missingno**

Permite a visualização fácil de missing values por coluna

In [ ]:
!pip install quilt

Instala pacote de dataset de exemplo:

In [ ]:
!quilt install ResidentMario/missingno_data

In [ ]:
from quilt.data.ResidentMario import missingno_data

Carrega a base de dados (base de dados de exemplo sobre as colisões na cidade de New York):

In [ ]:
df2 = missingno_data.nyc_collision_factors()

In [ ]:
df2.info()

In [ ]:
df2.isnull().sum()

### **Método Replace**


*   Substituição de dados a nível de dataframe
*   Substituição de dados usando regex


In [ ]:
teste = pd.DataFrame({'A': [0, 1, 2, 3, 4], 'B': [5, 6, 7, 8, 9], 'C': ['a', 'b', 'c', 'd', 'e']})

In [ ]:
teste

In [ ]:
teste.replace(0, 1000) # Se encontra o valor 0, substitui pelo valor 1000

In [ ]:
teste.replace([0, 1, 2, 3, 4], 'Novo Valor')

### **Substituição dos valores string 'nan' por tipo NaN:**

In [ ]:
df2.replace("nan", np.NaN, inplace=True)

In [ ]:
df2.isnull().sum()

In [ ]:
df2['ON STREET NAME'].value_counts(dropna=False) # Conta valores únicos, incluindo NaN

In [ ]:
import missingno as msno
msno.bar(df2.sample(1000)) # Quanto mais curta a barra, maior a quantidade de missing values da coluna

In [ ]:
msno.matrix(df2.sample(250)) # O que está cinza está preenchido

### **Remoção dos dados que não são úteis para a análise:**

Removendo colunas com 90% de dados missing:

In [ ]:
df2.shape[0] *.9 # Conta quantidade de linhas com missing

In [ ]:
df2.dropna(thresh=int(df2.shape[0] * .9), axis=1, inplace=True) # Se a coluna tiver o número de linhas passado para thresh, apaga a coluna do DataFrame

In [ ]:
%matplotlib inline
msno.matrix(df2.sample(250))

## **Data Cleaning em textos**


*   Tipo de dado em que é necessária maior limpeza, geralmente
*   Tarefas de pré-processamento adequadas para tipo de tarefas



Beautifier - toolkit para dados da web:

In [ ]:
!pip install beautifier

#### **Recursos para trabalhar com URL's:**

In [ ]:
from beautifier import Url
url = Url('https://in.linkedin.com/in/sachinphilip?authtoken=887nasdadasd6hasdtg21&secret=98jy766yhhuhnjk')
url.cleanup # Mostra a URL limpa

In [ ]:
url.domain # Mostra o domínio

In [ ]:
url.param # Mostra os parâmetros

#### **Recursos para trabalhar com dados de email:**

In [ ]:
from beautifier import Email
email = Email('me@insach.in')

In [ ]:
try:
  f = open("https://raw.githubusercontent.com/FabioDamacena/Pandas/main/email_test.txt", "r")
  for i in f.readlines():
    obj_email = Email(i)
    print(f'Username: {obj_email.username}, Domínio: {obj_email.domain}, Valido: {obj_email.is_valid}')
except:
  pass    

In [ ]:
try:
  f = open("https://raw.githubusercontent.com/FabioDamacena/Pandas/main/email_test.txt", "r")
  for i in f.readlines():
    obj_email = Email(i)
    if obj_email.is_free_email == False:
      print(f'Emails Corporativos: {i}') # Verifica tipo de domínio do email
except:
  pass

#### **Removendo pontuações:**

In [ ]:
import string
string.punctuation

In [ ]:
pontuacoes = string.punctuation
def remove_pontuacoes(text):
  return text.translate(str.maketrans('','', pontuacoes))

In [ ]:
remove_pontuacoes("!!!This is cool!!_*&%$#")

#### **Remoção de URL's**

In [ ]:
import re

In [ ]:
def remove_urls(text):
  url_padrao = re.compile(r'https?://\$+|www\.\$+')
  return url_padrao.sub(r'', text)

In [ ]:
print(remove_urls('O site do github é www.github.com ou https://github.com'))

#### **Remoção de tags HTML**

In [ ]:
def remove_html(texto):
  html_pattern = re.compile('<.*?.>')
  return html_pattern.sub(r'', texto)

In [ ]:
texto = """ <div>
<h1> Github</h1>
<p> Tutorial de uso do Python para Data Cleaning!</p>
<a href="https://github.com"> Repositorios de Data Science e de Machine Learning</a>
</div>"""

In [ ]:
print(remove_html(texto))

#### **Várias remoções em uma mesma função:**

In [ ]:
import string
import re


def text_cleaner(text):
  text = text.lower()
  pontuacoes = string.punctuation
  url_padrao = re.compile(r'https?://\$+|www\.\$+')
  html_padrao = re.compile('<.*?.>')
  text_temp = html_padrao.sub(r'', text)
  text_temp = url_padrao.sub(r'', text_temp)
  text_temp = text_temp.translate(str.maketrans('','', pontuacoes))
  return text_temp

In [ ]:
text_cleaner("<h1>O site do Github é https://github.com, cujas informações são muito úteis !!!</h1>")

'o site do github é httpsgithubcom cujas informações são muito úteis '

## **Versionamento de DataFrames**

Dora: Toolkit para análise exploratória de dados


*   Funciona no pandas e no scikit-learn
*   Contém métodos para simplificar tarefas
*   Cria versões de Dataframes



In [ ]:
!pip install Dora

### **Importação do Dora e carregamento do arquivo de dados:**

In [141]:
from Dora import Dora

In [142]:
df = missingno_data.nyc_collision_factors()

Estância de um objeto Dora:

In [143]:
dora = Dora(output = 0, data = df)

In [ ]:
type(dora.data)

In [ ]:
dora.data.head() #dora.[tecla TAB] mostra os diversos atributos do Dora

### **Criação de um snapshot:**

In [146]:
dora.snapshot('versao-0') # Cria uma versão inicial, caso queira salvar os dados sem as modificações

### **Remoção de colunas:**

In [147]:
dora.remove_feature('LOCATION') # Poderia usar dora.data.drop...

In [ ]:
dora.data.head()

In [ ]:
dora.logs # Mostra as alterações feitas no dora

### **Outras alterações:**

In [150]:
dora.data['LAT_LONG'] = dora.data.LATITUDE + dora.data.LONGITUDE # Criou uma coluna combinando latitute e longitude

In [ ]:
dora.data.head()

In [152]:
dora.snapshot('versao-1')

In [154]:
dora.data.drop(['NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED', 'CONTRIBUTING FACTOR VEHICLE 1'], axis=1, inplace=True)

In [ ]:
dora.data.head()

### **Utilizar o snapshot versão 1 e 0:**

In [160]:
dora.use_snapshot('versao-1')

In [ ]:
dora.data.head()

In [162]:
dora.use_snapshot('versao-0')

In [ ]:
dora.data.head()